In [4]:
import numpy as np
from scipy.special import ndtr
from scipy.stats import norm, ncx2
from scipy.optimize import minimize
from numpy.polynomial.hermite import hermfit, hermval, hermder
import copy
import fixed_income_derivatives as fid
import matplotlib.pyplot as plt

To calculate the credit default swaps (CDS), we use the following formula:

$$
C^* = \frac{LGD \cdot \sum_{j=n+1}^{N} p(t, T_j) \left[ \exp\left(-\int_{0}^{T_{j-1}} \lambda(s) ds\right) - \exp\left(-\int_{0}^{T_j} \lambda(s) ds\right) \right]}{\sum_{j=n+1}^{N} \alpha_j p(t, T_j) \exp\left(-\int_{0}^{T_j} \lambda(s) ds\right)}
$$

This function is defined in 'zcb_to_CDS'. The functions takes the usual inputs and also 'LGD' and 'intensity'. 'LGD' is difined as $RR = 1 - LGD$, where $RR$ is often 0.4. 'Intensity' are often not observed, but in this case, we provide an example of them. The code is:

In [5]:
def zcb_to_CDS(T_n,T_N,fixed_freq,T,p,LGD,intensity):
    I_n, idx_n = fid.value_in_list_returns_I_idx(T_n,T)
    I_N, idx_N = fid.value_in_list_returns_I_idx(T_N,T)

    if fixed_freq == "quarterly":
        alpha = 0.25
    elif fixed_freq == "semiannual":
        alpha = 0.5
    elif fixed_freq == "annual":
        alpha = 0.5

    D = 0
    for i in range(idx_n,idx_N):
        D += p[i]

    S = fid.zcb_to_accrual_factor(T_n,T_N,fixed_freq,T,p)

    intensity_exp_diff = np.exp(-sum(intensity[idx_n:idx_N])*alpha-sum(intensity[idx_n:idx_N+1])*alpha)
    intensity_exp = np.exp(-sum(intensity[idx_n:idx_N+1])*alpha)

    R = (LGD*D*intensity_exp_diff)/(S*intensity_exp)
    return R

RR = 0.4

LGD = 1 - RR

#Example:
T = [0, 0.5, 1, 1.5, 2, 2.5]
p = [1, 0.9, 0.8, 0.7, 0.5, 0.24]
intensity = [0.2, 0.3, 0.4, 0.3, 0.35, 0.123]

CDS = np.zeros(len(T))
for i in range(1,len(T)):
    CDS[i] = zcb_to_CDS(0,T[i],"semiannual",T,p,LGD,intensity)

CDS

array([0.        , 1.20644989, 1.04450929, 0.860798  , 0.7721212 ,
       0.68665404])

Instead, what we often observe are the credit default swaps on the market. Hence, we define an optimization function and fit the intensities. Doing this, we achieve the same intensities as above, EXCEPT for the last intensity (don't know why):

In [6]:
def fit_cds_obj(intensity,param,CDS_star,T,scaling = 1):
    p, LGD = param
    CDS_fit = np.zeros(len(T))

    for i in range(1,len(T)):
        CDS_fit[i] = zcb_to_CDS(0,T[i],"semiannual",T,p,LGD,intensity)

    M = len(T)
    y = 0
    for m in range(0,M):
        y += scaling*(CDS_fit[m] - CDS_star[m])**2
    return y


CDS_star = CDS 

print(CDS_star)
intensity_0 = [0.01, 0.4, 0.2, 0.1, 0.2, 0.1]

param = p, LGD

results = minimize(fit_cds_obj, intensity_0, args=(param,CDS_star,T), method='nelder-mead',  options={'xatol': 1e-8,'disp': True, 'maxiter': 100000})

print("Original intensity is: ", intensity)
print("Fitted intensities are: ", results.x)

[0.         1.20644989 1.04450929 0.860798   0.7721212  0.68665404]
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 499
         Function evaluations: 808
Original intensity is:  [0.2, 0.3, 0.4, 0.3, 0.35, 0.123]
Fitted intensities are:  [ 0.2         0.3         0.4         0.3         0.35       -0.85487954]
